In [1]:
from aux_train import *
from aux_plot import *
import plotly.graph_objects as go
from IPython.display import clear_output

In [2]:
import wandb
from wandb.keras import WandbCallback

In [3]:
# config = wandb.config
# config.equation='tranp_equation'
# config.WENO='WENO-Z-DS'

# config.poly_grade=7
# config.seno_ampli=3 
# config.gauss_var=3
# config.seizures=3

sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'validation_error',
      'goal': 'minimize'   
    },
    'parameters': {
        'poly_grade': {
            'values': list(range(8))
        },
        'seno_ampli': {
            'values': list(range(4))
        },
        'gauss_var': {
            'values': list(range(4))

        },
        'seizures': {
            'values': list(range(4))
        },
        'optimizer': {
            'values': ['Adam','RMSprop','Adamax','amsgrad','Adadelta','Ftrl']

        },
        'batch_size': {
            'values': [2**i for i in range(2,6)]
        },
        'learning_rate': {
            'distribution': 'log_uniform',
            'min': np.log(1e-5).tolist(),
            'max': np.log(1e-0).tolist()
        },
        'clip_norm': {
            'value': True
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project='WENO-DS', entity='silvaneojunior')
#sweep_id = 'silvaneojunior/LSTM Poemas/iuazw8n3'

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. learning_rate uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.


Create sweep with ID: ebdmyfkw
Sweep URL: https://wandb.ai/silvaneojunior/WENO-DS/sweeps/ebdmyfkw


In [4]:
default_equation='transp_equation'
default_WENO='WENO-Z-DS'

default_poly_grade=7
default_seno_ampli=3 
default_gauss_var=3
default_seizures=3

In [5]:
data_temporal,data_spatial,data_base,Δx,Δt,CFL,fronteira,equation=load_dataset(f'datasets/equation={default_equation} polys grau={default_poly_grade} seno_ampli={default_seno_ampli} gauss_var={default_gauss_var} seizure={default_seizures}')

indice = np.arange(data_temporal.shape[0])
np.random.shuffle(indice)
data_x = data_temporal.astype('float64')[indice]
data_y = data_temporal.astype('float64')[indice]

train_x = data_x[:,0:-1]
train_y = data_y[:,1:]

'Δx': 0.015625
'Δt': 0.004
'CFL': 0.5
'fronteira': <function FronteiraPeriodica at 0x0000015D86B7F550>
'equation': <class 'aux_equation.transp_equation'>
'seed': None



In [6]:
Sim_Z=simulation(API_Numpy,equation,WENO_Z)
Δx_sol=0.01
Δt_sol=Δx_sol*CFL
t_final=0.2
total_time=int(t_final/Δt_sol)

In [7]:
f_test=f_test_1
x,fx=create_f_points(f_test=f_test,Δx=Δx_sol)
x_dense,fx_dense=create_f_points(f_test=f_test,Δx=Δx_sol/10)

fxs_list=[fx]
fxs_dense_list=[fx_dense]
for i in range(total_time):
    print(i,end='\r')
    fxs_dense_list.append(Sim_Z.Sim(fxs_dense_list[-1],Δt_sol,Δx_sol/10,CFL,FronteiraPeriodica))
    fxs_list.append(Sim_Z.Sim(fxs_list[-1],Δt_sol,Δx_sol,CFL,FronteiraPeriodica))
    

In [8]:
fxs=np.stack(fxs_list,axis=0)#[50:]
fxs_dense=np.stack(fxs_dense_list,axis=0)#[50:]
total_time=fxs.shape[0]-1

In [9]:
# plot_all_registry=[]
# for i in range(total_time):
#     fig=go.Figure()
#     fig.add_trace(go.Scatter(x=x_dense, y=fxs_dense[i+1,0],
#                         mode='lines',
#                         name='Sol. ref.'))
#     fig.add_trace(go.Scatter(x=x, y=fxs[i+1,0],
#                         mode='lines',
#                         name='WENO-Z'))
#     plot_all_registry.append(fig)

In [10]:
def train():
    clear_output()
    wandb.init()
    config = wandb.config

    # default_config = wandb.config
    # default_config.equation='tranp_equation'
    # default_config.WENO='WENO-Z-DS'

    # default_config.poly_grade=7
    # default_config.seno_ampli=3 
    # default_config.gauss_var=3
    # default_config.seizures=3

    # config = default_config

    run_name = wandb.run.name

    equation_name=default_equation
    poly_grade=config.poly_grade
    seno_ampli=config.seno_ampli
    gauss_var=config.gauss_var
    seizures=config.seizures

    
    optimizer=config.optimizer
    batch_size=config.batch_size
    learning_rate=config.learning_rate
    clip_norm=config.clip_norm

    print(f'equation_name={equation_name}')
    print(f'poly_grade={poly_grade}')
    print(f'seno_ampli={seno_ampli}')
    print(f'gauss_var={gauss_var}')
    print(f'seizures={seizures}')
    print(f'optimizer={optimizer}')
    print(f'batch_size={batch_size}')
    print(f'learning_rate={learning_rate}')
    print(f'clip_norm={clip_norm}')

    data_temporal,data_spatial,data_base,Δx,Δt,CFL,fronteira,equation=load_dataset(f'datasets/equation={equation_name} polys grau={poly_grade} seno_ampli={seno_ampli} gauss_var={gauss_var} seizure={seizures}')

    indice = np.arange(data_temporal.shape[0])
    np.random.shuffle(indice)
    data_x = data_temporal.astype('float64')[indice]
    data_y = data_temporal.astype('float64')[indice]

    train_x = data_x[:,0:-1]
    train_y = data_y[:,1:]

    Sim_layer = WENO_temporal_layer(equation,WENO_Z,Δx=Δx,Δt=Δt,fronteira=fronteira,regul_weight=0,ativ_func=tf.nn.sigmoid,p=2,conv_size=5)

    input_x   = keras.layers.Input(train_x.shape[1:], dtype='float64')
    if optimizer=='Adam':
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=clip_norm)
    elif optimizer=='RMSprop':
        optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate, clipnorm=clip_norm)
    elif optimizer=='Adamax':
        optimizer = keras.optimizers.Adamax(learning_rate=learning_rate, clipnorm=clip_norm)
    elif optimizer=='amsgrad':
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate, amsgrad=True, clipnorm=clip_norm)
    elif optimizer=='Adadelta':
        optimizer = keras.optimizers.Adadelta(learning_rate=learning_rate, clipnorm=clip_norm)
    elif optimizer=='Ftrl':
        optimizer = keras.optimizers.Ftrl(learning_rate=learning_rate, clipnorm=clip_norm)
    
    output_x=input_x

    #layers_norm=keras.layers.BatchNormalization()
    #output_x=layers_norm(output_x)
    output_x=keras.layers.TimeDistributed(Sim_layer)(output_x)

    # Criando a rede neural
    Network = keras.Model(input_x, output_x)
    Network.compile(loss=MES_relative(Δx/2), optimizer=optimizer, metrics=['mean_absolute_error'])

    Network.fit(
                train_x                           , # Dados de treino
                train_y                           , # Dados de treino
                batch_size      = batch_size               , # Tamanho do batch
                epochs          = 70              , # Número de epochs
                steps_per_epoch = 10              , # Número de epochs
                shuffle         = True            ,  # Aleatorização dos batchs
                callbacks=[WandbCallback(training_data=(train_x,train_y),log_weights=True,log_gradients=True)]
            )
        #wandb.log({"loss": loss, "epoch": epoch})

    if not(os.path.isdir(f'Modelos treinados/{sweep_id}/')):
        os.mkdir(f'Modelos treinados/{sweep_id}/')
    save_model(Network,Sim_layer.config,f'Modelos treinados/{sweep_id}/{run_name}')

    prev_fx_list=[fxs[0]]
    count=0
    for i in range(total_time):
        if(i%10==0):
            print(count,end='\r')
        prev_fx_list.append(Sim_layer.Sim_step(prev_fx_list[-1],Δt_sol, Δx_sol, FronteiraPeriodica))
        # plot_all_registry[i].add_trace(go.Scatter(x=x, y=prev_fx_list[-1][0],
        #                 mode='lines',
        #                 name=run_name))

        # fig=go.Figure()
        # fig.add_trace(go.Scatter(x=x_dense, y=fxs_dense[i+1,0],
        #                     mode='lines',
        #                     name='Sol. ref.'))
        # fig.add_trace(go.Scatter(x=x, y=fxs[i+1,0],
        #                     mode='lines',
        #                     name='WENO-Z'))        
        # fig.add_trace(go.Scatter(x=x, y=prev_fx_list[-1][0],
        #                 mode='lines',
        #                 name=run_name))

        # wandb.log({'previsão_fx':plot_all_registry[i],'previsão_fx_solo':fig},step=i)
        count+=Δt_sol

    prev_fx=np.stack(prev_fx_list[1:],axis=0)
    ref_fx=fxs_dense[1:prev_fx.shape[0]+1,:,range(0,fxs_dense.shape[-1],10)]

    #prev_fx=Sim_layer.Sim(fx,2*(3), 0.01, CFL, FronteiraPeriodica)
    erro=2*tf.math.reduce_mean(tf.abs(prev_fx-ref_fx))/Δx_sol
    
    wandb.log({'validation_error':erro})

#wandb.agent(sweep_id, train)

In [11]:
wandb.agent(sweep_id, train)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


equation_name=transp_equation
poly_grade=7
seno_ampli=1
gauss_var=2
seizures=2
optimizer=Ftrl
batch_size=4
learning_rate=0.7886664797459279
clip_norm=True
'Δx': 0.015625
'Δt': 0.004
'CFL': 0.5
'fronteira': <function FronteiraPeriodica at 0x0000015D86B7F550>
'equation': <class 'aux_equation.transp_equation'>
'seed': None

Epoch 1/70
10/10 [==============================] - 60s 5s/step - loss: 0.0177 - mean_absolute_error: 0.0012 - _timestamp: 1651613231.0000 - _runtime: 93.0000
Epoch 2/70
10/10 [==============================] - 30s 3s/step - loss: 0.0142 - mean_absolute_error: 0.0011 - _timestamp: 1651613261.0000 - _runtime: 123.0000
Epoch 3/70
10/10 [==============================] - 30s 3s/step - loss: 0.0197 - mean_absolute_error: 0.0013 - _timestamp: 1651613291.0000 - _runtime: 153.0000
Epoch 4/70
10/10 [==============================] - 31s 3s/step - loss: 0.0135 - mean_absolute_error: 9.8269e-04 - _timestamp: 1651613322.0000 - _runtime: 184.0000
Epoch 5/70
10/10 [================

epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██
loss,▅▃▆▂▃▅▄▅▄▁▄▅▃█▃▂▂▃▃▆▃▄
mean_absolute_error,▅▂▆▁▄▄▃▆▄▁▃▄▂█▃▃▁▃▄▅▁▄
epoch,21
loss,0.01585
mean_absolute_error,0.00116
